# Differential evolution
In this notebook, we will explore the Differential Evolution (DE) algorithm. We'll explain its purpose, how it works, and demonstrate it using example functions and visualizations.

## What is Differential evolution
Differential Evolution is a population-based stochastic optimization algorithm inspired by natural evolution. It evolves a population of candidate solutions using operations like mutation, crossover, and selection, aiming to find the global minimum (or maximum) of a function.

**Key points:**
   - Population-based: maintains multiple candidate solutions simultaneously.
   - Stochastic: uses randomness in mutation and crossover to explore the search space.
   - Suitable for continuous, nonlinear, and non-differentiable problems.

**Stops if:**
   - maximum generations (cycles) are reached, or
   - the population has converged (no significant improvement), or
   - the maximum number of function evaluations is reached.

**Notes:**
   - More robust than simple local search methods because it explores multiple solutions at once.
   - Can handle complex landscapes with multiple local minima.
   - Parameters like population size, mutation factor, and crossover probability affect performance.

## How Differential Evolution works
1. Initialize the population: generate a set of candidate solutions uniformly within the search bounds.
2. Repeat for each generation:
    - Mutation: for each individual, create a mutant vector by combining other randomly selected individuals.
    - Crossover: mix the mutant vector with the current individual to create a trial vector.
    - Selection: if the trial vector performs better than the current individual (based on the objective function), replace the current individual with the trial.
3. Record the population at each generation for analysis or visualization.
4. Return the history of population states over generations.

## How does Differential Evolutionlook?
You can check the implementation here or inside 'src/algorithms/differential_evolution.py'.

```python
class DifferentialEvolution(Algorithm):
    """
    Differential Evolution optimization algorithm.

    This population-based stochastic optimization method evolves a population
    of candidate solutions using operations inspired by natural evolution:
    mutation, crossover, and selection.

    Each generation produces new trial solutions by combining existing ones,
    and individuals are replaced by better-performing offspring based on
    the provided objective function.

    The algorithm is suitable for continuous, nonlinear, and non-differentiable
    optimization problems.
    """

    def __init__(
            self,
            lower_bound,
            upper_bound,
            function,
            iterations=10_000,
            individuals=10,
            mutation=.5,
            crossover=.5,
            cycles=100
        ):
        """
        Initialize the DifferentialEvolution algorithm.

        Args:
            lower_bound (float): Lower bound of the search space.
            upper_bound (float): Upper bound of the search space.
            function (callable): Objective function to minimize.
                                 Must accept a NumPy array and return a scalar value.
            iterations (int, optional): Total number of function evaluations allowed (default is 10,000).
            individuals (int, optional): Number of individuals (population size) (default is 10).
            mutation (float, optional): Mutation factor (scales the differential variation) (default is 0.5).
            crossover (float, optional): Probability of crossover between individuals (default is 0.5).
            cycles (int, optional): Number of generations (iterations of evolution) (default is 100).
        """
        super().__init__(lower_bound, upper_bound, function, iterations)
        self.individuals = individuals
        self.mutation = mutation
        self.crossover = crossover
        self.cycles = cycles

    @staticmethod
    def generate_population(lower_bound, upper_bound, input_np, dimension=2):
        """
        Generate an initial population of individuals.

        Each individual is a vector sampled uniformly within the search bounds.

        Args:
            lower_bound (float): Lower bound of the search space.
            upper_bound (float): Upper bound of the search space.
            input_np (int): Number of individuals in the population.
            dimension (int, optional): Dimensionality of each individual (default is 2).

        Returns:
            list of list[float]: A list containing the initial population,
                                 where each individual is represented as a list of floats.
        """
        return [np.random.uniform(lower_bound, upper_bound, dimension).tolist()
                for _ in range(input_np)]

    @staticmethod
    def get_random_parents(population, exclude):
        """
        Select a random individual index from the population,
        excluding specific individuals.

        Args:
            population (list): Current list of individuals.
            exclude (list): List of individuals to exclude from selection.

        Returns:
            int: Index of a randomly chosen individual not in the exclude list.
        """
        result = [i for i in range(len(population)) if population[i] not in exclude]
        return np.random.choice(result)

    def run(self):
        """
        Run the Differential Evolution optimization process.

        This method executes the full evolutionary loop:
        - Initializes a population of candidate solutions.
        - Iteratively applies mutation, crossover, and selection.
        - Retains better-performing individuals over generations.

        Returns:
            list of list[list[float]]: History of population states over generations.
                                       Each element represents a generation, which is
                                       a list of individuals (each being a list of coordinates).
        """
        # History of all generations
        history = []

        # Initialize population
        pop = self.generate_population(self.lower_bound, self.upper_bound,
                                       self.individuals, dimension=2)

        for g in range(self.cycles):
            new_population = copy.deepcopy(pop)

            for i, individual in enumerate(pop):
                # Select three distinct random parents different from the current individual
                r1_i = int(self.get_random_parents(pop, [individual]))
                r2_i = int(self.get_random_parents(pop, [individual, pop[r1_i]]))
                r3_i = int(self.get_random_parents(pop, [individual, pop[r1_i], pop[r2_i]]))

                # Fetch parent vectors
                r1 = np.array(new_population[r1_i])
                r2 = np.array(new_population[r2_i])
                r3 = np.array(new_population[r3_i])

                # Mutation: create a mutant vector
                mutated = (r1 - r2) * self.mutation + r3

                # Crossover: create a trial vector
                trial = np.zeros(len(individual))
                j_rnd = np.random.randint(0, len(individual))  # at least one gene from mutant

                for j in range(len(individual)):
                    if np.random.uniform() < self.crossover or j == j_rnd:
                        trial[j] = mutated[j]
                    else:
                        trial[j] = individual[j]

                # Enforce bounds
                trial = np.clip(trial, self.lower_bound, self.upper_bound)

                # Selection: accept the trial if it performs better
                if self.function(np.array(trial)) <= self.function(np.array(individual)):
                    new_population[i] = list(trial)

            # Record generation
            history.append(copy.deepcopy(new_population))
            pop = new_population

        return history

```

## Visualization
Next, we will visualize the Hill Climbing algorithm using the following benchmark functions:

- Sphere
- Ackley
- Rastrigin
- Rosenbrock
- Griewank
- Schwefel
- Lévy
- Michalewicz
- Zakharov

For more details about these functions and their implementations, please refer to the `test_functions.ipynb` notebook or check the `src/functions.py` file.

In [1]:
from src.benchmark_algorithm import benchmark_algorithm
from src.algorithms.differential_evolution import DifferentialEvolution

benchmark_algorithm(DifferentialEvolution)

Function: sphere, Algorithm: DifferentialEvolution, Best found solution: 6.703796624533959e-26
Animating step #0
Animating step #0
Animating step #1
Animating step #2
Animating step #3
Animating step #4
Animating step #5
Animating step #6
Animating step #7
Animating step #8
Animating step #9
Animating step #10
Animating step #11
Animating step #12
Animating step #13
Animating step #14
Animating step #15
Animating step #16
Animating step #17
Animating step #18
Animating step #19
Animating step #20
Animating step #21
Animating step #22
Animating step #23
Animating step #24
Animating step #25
Animating step #26
Animating step #27
Animating step #28
Animating step #29
Animating step #30
Animating step #31
Animating step #32
Animating step #33
Animating step #34
Animating step #35
Animating step #36
Animating step #37
Animating step #38
Animating step #39
Animating step #40
Animating step #41
Animating step #42
Animating step #43
Animating step #44
Animating step #45
Animating step #46
Anim

### Optimization Animation

Here is the Hill climbing process visualized:

<table>
<tr>
  <td><img src="../assets/DifferentialEvolution/sphere.gif" width="500"/></td>
  <td><img src="../assets/DifferentialEvolution/ackley.gif" width="500"/></td>
  <td><img src="../assets/DifferentialEvolution/rastrigin.gif" width="500"/></td>
</tr>
<tr>
  <td><img src="../assets/DifferentialEvolution/rosenbrock.gif" width="500"/></td>
  <td><img src="../assets/DifferentialEvolution/griewank.gif" width="500"/></td>
  <td><img src="../assets/DifferentialEvolution/schwefel.gif" width="500"/></td>
</tr>
<tr>
  <td><img src="../assets/DifferentialEvolution/levy.gif" width="500"/></td>
  <td><img src="../assets/DifferentialEvolution/michalewicz.gif" width="500"/></td>
  <td><img src="../assets/DifferentialEvolution/zakharov.gif" width="500"/></td>
</tr>
</table>

## Summary
- Differential Evolution is a stochastic, population-based optimization algorithm.
- Efficient for continuous and complex landscapes with multiple local optima.
- Maintains diversity in the population, reducing the risk of getting stuck in local minima.
- Performance depends on proper choice of population size, mutation factor, crossover probability, and number of generations.
- Benchmark functions and visualizations help analyze its behavior and convergence.